# Chapter 8: Finding Patterns - Market Basket Analysis Using Association Rules
Machine Learning with R - Brett Lantz 

---



Impulse purchases are often no coincidence as retailers use refined techniques to analyze customer buying behavior. Using this example, Chapter 8 describes and explains how machine learning algorithms are able to search for purchasing patterns of retail customers using barcode scanners, inventory databases and online shopping cards. This search is commonly referred to as market basket analysis. 


## Theoretical basics
This part aims to provide the reader with an theoretical understanding of the underlying concepts of market basket analysis.
### Understanding association rules
The goal of market basket analyses is the generation of association rules which enumerate patterns within the relationship among (purchased) items. An itemset is a group of one or more items in a single transaction and is denoted as follows: 
> {bread, peanut butter, jelly}.

Resulting from this, the denotation of association rules is

> {peanut butter,jelly} → {bread}. .

The left-hand-side (LHS) is the condition that has to be met in order to trigger the rule and the right-hand-side (RHS) is the expected result of meeting that condition.  
Association rules are unsupervised learners which means that there is no need for the algorithm to be trained as it only finds interesting patterns in the dataset. Furthermore, they are used for knowledge discovery in large datasets rather than predictions. The foremost downside of association rules is the absence of an objective way to measure performance rule learner. 

### The apriori algorithm
The datasets used for market basket analyses are almost always large. The number of potential itemsets grows exponentially with the number of features. The apriori algorithm constitutes a solution for this dilemma as the algorithm utilizes a simple prior belief about the properties of frequent itemsets. The apriori property suggests that all subsets of a frequent itemset must also be frequent: If either {motor oil} or {lipstick} or both are considered infrequent the itemset {motor oil, lipstick} is eliminated as well. This approach reduces the number of rules for which the algorithm searches.

The algorithm uses statistical measures, namely support, confidence and lift, of an itemset to define a minimum threshold and to locate association rules. The **support** of an itemset answers the question of how frequently an itemset or rule occur in the data. 
A rule’s **confidence** is the measurement of its predictive power/accuracy and is defined with the following formula: 

> confidence (X→Y)=(support(X,Y))/(support(X)).

Important is that the confidence that X leads to Y is not the same as confidence that Y leads to X. Strong association rules have both high support and high confidence. 
The metric **lift** measures how much more likely one item or itemset is to be purchased relative to its typical rate of purchase, given that you know another item or itemset has been purchased. The according formula is:

> lift(X→Y)=(confidence(X→Y))/(support (Y)).

The procedure of the apriori algorithm is as follows:

1.   The algorithm identifies all itemsets that meet the minimum support threshold applying multiple iterations with increasing size of itemsets.
2.   Then, the algorithm creates rules from these itemsets using those meeting the minimums confidence threshold.


 





## Example - identifying frequently purchased groceries with association rules

###Step 1: Collecting data
The dataset comprises of 9835 transactions and has is based on the fundamental assumption that only one specific brand per product is offered. 

###Step 2: Exploring and preparing the data
The transactional data used in this chapter differs from the one used in previous chapters. The dataset is comprised of comma-separated lists of varying number of items purchased in each transaction. Instead of using a conventional matrix where each row has the same set of features a sparse matrix is used. Each row of the latter contains a transaction and runs through as many columns as there are possible items in the shopping bag. As not every customer buys all product types, some cells doesn’t contain values. This characteristic makes sparse matrices memory efficient as only occupied cells are saved. 

In [0]:
install.packages('arules', repos='http://cran.rstudio.com/')
library(arules)
groceries <- read.transactions("groceries.csv", sep = ",")
summary(groceries)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in file(con, "r"):
“cannot open file 'groceries.csv': No such file or directory”


ERROR: ignored